# Uniform Quantization Tables & Figures

Tables IV & V + Figures for the paper.  
**How to update:** Edit the `DATA` dict in Cell 1, then Run All.  
Figures are saved to `figures/` folder.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
matplotlib.rcParams.update({'font.size': 11, 'font.family': 'serif'})

FIG_DIR = os.path.join(os.getcwd(), 'figures')
os.makedirs(FIG_DIR, exist_ok=True)

# =================================================================
# DATA — Update values here, then Run All
# =================================================================
ENC_PARAMS = {
    'CsiNet':                {'cr4': 1_049_254, 'cr16': 262_438},
    'CLNet':                 {'cr4': 1_049_556, 'cr16': 263_253},
    'TransNet':              {'cr4': None,      'cr16': None},
    'Mamba-Transformer AE':  {'cr4': 1_091_586, 'cr16': None},
}

# NMSE (dB): {model: {cr: {weight_bits: nmse_db}}}
# None = not yet measured
NMSE = {
    'CsiNet': {
        'cr4':  {16: -8.74,  8: 1.46,   4: 19.40,  2: 22.32},
        'cr16': {16: -4.51,  8: -4.31,  4: 16.74,  2: 22.54},
    },
    'CLNet': {
        'cr4':  {16: -12.82, 8: 0.15,   4: 23.36,  2: 25.75},
        'cr16': {16: -5.55,  8: 13.62,  4: 25.66,  2: 22.63},
    },
    'TransNet': {
        'cr4':  {16: None, 8: None, 4: None, 2: None},
        'cr16': {16: None, 8: None, 4: None, 2: None},
    },
    'Mamba-Transformer AE': {
        'cr4':  {16: -15.37, 8: -15.19, 4: 0.03,   2: 7.69},
        'cr16': {16: None,   8: None,   4: None,    2: None},
    },
}

# Enc. BOPs override (when enc_params not available, use measured values)
# {model: {cr: {weight_bits: bops_in_M}}}
BOPS_OVERRIDE = {
    'Mamba-Transformer AE': {
        'cr4': {16: 1245.46, 8: 622.85, 4: 311.47, 2: 155.78},
    },
}

ACT_BITS = 16
WEIGHT_BITS_LIST = [16, 8, 4, 2]
MODELS = list(NMSE.keys())

def calc_bops(model, cr_key, w_bits):
    # Check override first
    if model in BOPS_OVERRIDE and cr_key in BOPS_OVERRIDE[model]:
        val = BOPS_OVERRIDE[model][cr_key].get(w_bits)
        if val is not None: return val * 1e6
    enc_p = ENC_PARAMS[model][cr_key]
    if enc_p is None: return None
    return enc_p * w_bits * ACT_BITS

def calc_saving(w_bits):
    return (1 - (w_bits * ACT_BITS) / (32 * 32)) * 100

MARKERS = {'CsiNet': 's', 'CLNet': 'D', 'TransNet': '^', 'Mamba-Transformer AE': 'o'}
COLORS  = {'CsiNet': '#1f77b4', 'CLNet': '#ff7f0e', 'TransNet': '#2ca02c', 'Mamba-Transformer AE': '#d62728'}

print('Data loaded. Models:', MODELS)

Data loaded. Models: ['CsiNet', 'CLNet', 'TransNet', 'Mamba-Transformer AE']


## Table IV — Uniform Quantization (CR=1/4)

In [2]:
def build_table(cr_key):
    rows = []
    for model in MODELS:
        for wb in WEIGHT_BITS_LIST:
            nmse = NMSE[model][cr_key].get(wb)
            bops = calc_bops(model, cr_key, wb)
            saving = calc_saving(wb)
            rows.append({
                'Model': model,
                'Prec.': f'INT{wb}',
                'NMSE (dB)': f'{nmse:.2f}' if nmse is not None else '---',
                'Enc. BOPs (M)': f'{bops/1e6:.2f}' if bops is not None else '---',
                'Saving vs. FP32 (%)': f'{saving:.2f}',
            })
    return pd.DataFrame(rows)

df4 = build_table('cr4')
print('TABLE IV: Uniform Quantization Performance (CR=1/4)')
print('Encoder activations fixed to INT16; listed precision = weight bit-width.\n')
print(df4.to_string(index=False))

TABLE IV: Uniform Quantization Performance (CR=1/4)
Encoder activations fixed to INT16; listed precision = weight bit-width.

               Model Prec. NMSE (dB) Enc. BOPs (M) Saving vs. FP32 (%)
              CsiNet INT16     -8.74        268.61               75.00
              CsiNet  INT8      1.46        134.30               87.50
              CsiNet  INT4     19.40         67.15               93.75
              CsiNet  INT2     22.32         33.58               96.88
               CLNet INT16    -12.82        268.69               75.00
               CLNet  INT8      0.15        134.34               87.50
               CLNet  INT4     23.36         67.17               93.75
               CLNet  INT2     25.75         33.59               96.88
            TransNet INT16       ---           ---               75.00
            TransNet  INT8       ---           ---               87.50
            TransNet  INT4       ---           ---               93.75
            TransNet  

## Table V — Uniform Quantization (CR=1/16)

In [3]:
df16 = build_table('cr16')
print('TABLE V: Uniform Quantization Performance (CR=1/16)')
print('Encoder activations fixed to INT16; listed precision = weight bit-width.\n')
print(df16.to_string(index=False))

TABLE V: Uniform Quantization Performance (CR=1/16)
Encoder activations fixed to INT16; listed precision = weight bit-width.

               Model Prec. NMSE (dB) Enc. BOPs (M) Saving vs. FP32 (%)
              CsiNet INT16     -4.51         67.18               75.00
              CsiNet  INT8     -4.31         33.59               87.50
              CsiNet  INT4     16.74         16.80               93.75
              CsiNet  INT2     22.54          8.40               96.88
               CLNet INT16     -5.55         67.39               75.00
               CLNet  INT8     13.62         33.70               87.50
               CLNet  INT4     25.66         16.85               93.75
               CLNet  INT2     22.63          8.42               96.88
            TransNet INT16       ---           ---               75.00
            TransNet  INT8       ---           ---               87.50
            TransNet  INT4       ---           ---               93.75
            TransNet  

## Figure — NMSE vs. BOPs Saving (solid: CR=1/4, dashed: CR=1/16)

In [4]:
fig, ax = plt.subplots(figsize=(6, 4.5))

LINE_STYLES = {'cr4': '-', 'cr16': '--'}
CR_LABELS   = {'cr4': 'CR=1/4', 'cr16': 'CR=1/16'}

for model in MODELS:
    for cr_key in ['cr4', 'cr16']:
        savings, nmses = [], []
        for wb in WEIGHT_BITS_LIST:
            nmse = NMSE[model][cr_key].get(wb)
            if nmse is None: continue
            savings.append(calc_saving(wb))
            nmses.append(nmse)
        if not savings: continue
        ax.plot(savings, nmses, marker=MARKERS[model], color=COLORS[model],
                linestyle=LINE_STYLES[cr_key], linewidth=1.8, markersize=7,
                label=f'{model}, {CR_LABELS[cr_key]}')

ax.set_xlabel('BOPs Saving vs. FP32 (%)')
ax.set_ylabel('NMSE (dB)')
ax.legend(fontsize=7.5, loc='upper left', ncol=1)
ax.grid(True, linestyle='--', alpha=0.5)
ax.set_xlim(70, 100)
plt.tight_layout()
fig.savefig(os.path.join(FIG_DIR, 'fig_uniform_quant.pdf'), bbox_inches='tight', dpi=300)
fig.savefig(os.path.join(FIG_DIR, 'fig_uniform_quant.png'), bbox_inches='tight', dpi=300)
print('Saved: figures/fig_uniform_quant.pdf / .png')
plt.show()

Saved: figures/fig_uniform_quant.pdf / .png


c:\users\박현재\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:28: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.


## LaTeX Source (copy-paste to paper)

In [5]:
def gen_latex(cr_key, cr_label):
    lines = []
    lines.append(r'\begin{table}[t]')
    lines.append(r'\centering')
    lines.append(r'\caption{Uniform Quantization Performance ($\mathrm{CR}=' + cr_label + r'$); encoder activations are fixed to INT16 and the listed precision refers to weight bit-width.}')
    lines.append(r'\label{tab:uniform_quant_' + cr_key + r'}')
    lines.append(r'\setlength{\tabcolsep}{4pt}')
    lines.append(r'\begin{tabular}{llccc}')
    lines.append(r'\toprule')
    lines.append(r'Model & Prec. & NMSE (dB) & Enc.\ BOPs (M) & Saving vs.\ FP32 (\%) \\\\')
    lines.append(r'\midrule')
    for i, model in enumerate(MODELS):
        for j, wb in enumerate(WEIGHT_BITS_LIST):
            nmse = NMSE[model][cr_key].get(wb)
            bops = calc_bops(model, cr_key, wb)
            saving = calc_saving(wb)
            model_col = model if j == 0 else ''
            nmse_str = f'{nmse:.2f}' if nmse is not None else '---'
            bops_str = f'{bops/1e6:.2f}' if bops is not None else '---'
            lines.append(f'{model_col} & INT{wb} & {nmse_str} & {bops_str} & {saving:.2f} \\\\\\\\')
        if i < len(MODELS) - 1:
            lines.append(r'\midrule')
    lines.append(r'\bottomrule')
    lines.append(r'\end{tabular}')
    lines.append(r'\end{table}')
    return '\n'.join(lines)

print('% === TABLE IV ===')
print(gen_latex('cr4', '1/4'))
print()
print('% === TABLE V ===')
print(gen_latex('cr16', '1/16'))

% === TABLE IV ===
\begin{table}[t]
\centering
\caption{Uniform Quantization Performance ($\mathrm{CR}=1/4$); encoder activations are fixed to INT16 and the listed precision refers to weight bit-width.}
\label{tab:uniform_quant_cr4}
\setlength{\tabcolsep}{4pt}
\begin{tabular}{llccc}
\toprule
Model & Prec. & NMSE (dB) & Enc.\ BOPs (M) & Saving vs.\ FP32 (\%) \\\\
\midrule
CsiNet & INT16 & -8.74 & 268.61 & 75.00 \\\\
 & INT8 & 1.46 & 134.30 & 87.50 \\\\
 & INT4 & 19.40 & 67.15 & 93.75 \\\\
 & INT2 & 22.32 & 33.58 & 96.88 \\\\
\midrule
CLNet & INT16 & -12.82 & 268.69 & 75.00 \\\\
 & INT8 & 0.15 & 134.34 & 87.50 \\\\
 & INT4 & 23.36 & 67.17 & 93.75 \\\\
 & INT2 & 25.75 & 33.59 & 96.88 \\\\
\midrule
TransNet & INT16 & --- & --- & 75.00 \\\\
 & INT8 & --- & --- & 87.50 \\\\
 & INT4 & --- & --- & 93.75 \\\\
 & INT2 & --- & --- & 96.88 \\\\
\midrule
Mamba-Transformer AE & INT16 & -15.37 & 1245.46 & 75.00 \\\\
 & INT8 & -15.19 & 622.85 & 87.50 \\\\
 & INT4 & 0.03 & 311.47 & 93.75 \\\\
 & INT2 &